# Lab 02: Dates & File I/O (CSV & Excel) — ~30 minutes

**Dataset:** `SampleData.csv`

By the end, you'll be able to:
- Load CSV files and parse date columns.
- Work with `datetime` features (year, month, quarter).
- Filter and aggregate by date.
- Write data to **CSV** and **Excel** (multiple sheets), and read it back.


## Prerequisites
- Python 3.9+ and `pandas` installed (`pip install pandas`)
- For Excel writing/reading: `openpyxl` (`pip install openpyxl`)
- Place the provided `SampleData.csv` in the same folder as this notebook.

> If you're not sure you have the packages:
>
> ```bash
> pip install pandas openpyxl
> ```


## Part 0 — Setup (2 minutes)
Create an `outputs/` folder to hold files you'll write.

In [1]:
from pathlib import Path
Path("outputs").mkdir(exist_ok=True)
print("Created ./outputs (if not already present)")

Created ./outputs (if not already present)


## Part 1 — Read CSV & parse dates (5 minutes)
1) **Load** the CSV and parse `Invoice_date` as `datetime`.
2) **Sanity checks**: Check types, head, and min/max dates.

In [2]:
import pandas as pd

df = pd.read_csv(
    "SampleData.csv",
    parse_dates=["Invoice_date"]  # ensures correct dtype on read
)

print(df.dtypes)
display(df.head())
df.info()
print("Min/Max Invoice_date:", df["Invoice_date"].min(), df["Invoice_date"].max())

Account_no                   int64
LocationID                  object
CustomerID                  object
ProductID                   object
Billed_usage_kwh             int64
Invoice_date        datetime64[ns]
Base_charge                float64
Price                      float64
Bill                       float64
dtype: object


,Account_no,LocationID,CustomerID,ProductID,Billed_usage_kwh,Invoice_date,Base_charge,Price,Bill
0,1002001,L1001,P01,P01,302,2020-03-05,12.0,0.03,9.06
1,1002001,L1001,P01,P01,520,2020-04-06,12.0,0.03,15.60
2,1002001,L1001,P01,P01,619,2020-05-04,12.0,0.03,18.57
3,1002001,L1001,P01,P01,614,2020-06-03,12.0,0.03,18.42
4,1002001,L1001,P01,P01,1389,2020-07-03,12.0,0.03,41.67


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Account_no        288 non-null    int64         
 1   LocationID        288 non-null    object        
 2   CustomerID        288 non-null    object        
 3   ProductID         288 non-null    object        
 4   Billed_usage_kwh  288 non-null    int64         
 5   Invoice_date      288 non-null    datetime64[ns]
 6   Base_charge       288 non-null    float64       
 7   Price             288 non-null    float64       
 8   Bill              288 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 20.4+ KB
Min/Max Invoice_date: 2020-03-02 00:00:00 2021-02-01 00:00:00


## Part 2 — Work with dates (8 minutes)
Add common calendar features, set a `DatetimeIndex` for easier filtering, and create a month-normalized column.

In [3]:
# Add calendar features
df["Year"] = df["Invoice_date"].dt.year
df["Month"] = df["Invoice_date"].dt.month
df["MonthName"] = df["Invoice_date"].dt.month_name()
df["Quarter"] = df["Invoice_date"].dt.quarter
df["InvoiceMonth"] = df["Invoice_date"].dt.to_period("M").dt.to_timestamp("M")  # month end

# Set a DatetimeIndex (copy)
df_dt = df.set_index("Invoice_date").sort_index()
display(df_dt.head(3))

,Account_no,LocationID,CustomerID,ProductID,Billed_usage_kwh,Base_charge,Price,Bill,Year,Month,MonthName,Quarter,InvoiceMonth
Invoice_date,,,,,,,,,,,,,
2020-03-02,1002024,L1024,P03,P03,690,12.0,0.025,17.25,2020,3,March,1,2020-03-31
2020-03-02,1002006,L1006,P03,P03,216,12.0,0.025,5.40,2020,3,March,1,2020-03-31
2020-03-02,1002007,L1007,P01,P01,527,12.0,0.030,15.81,2020,3,March,1,2020-03-31


In [4]:
# Filter by date range (e.g., July–December 2020)
mask = (df_dt.index >= "2020-07-01") & (df_dt.index <= "2020-12-31")
df_jul_dec_2020 = df_dt.loc[mask]
display(df_jul_dec_2020[["Account_no","CustomerID","Billed_usage_kwh","Price","Bill"]].head())

# Alternate filter using the InvoiceMonth column
display(df[df["InvoiceMonth"].between("2020-07-31", "2020-12-31")].head(3))

,Account_no,CustomerID,Billed_usage_kwh,Price,Bill
Invoice_date,,,,,
2020-07-01,1002014,P02,1239,0.045,55.75
2020-07-01,1002012,P03,1505,0.025,37.62
2020-07-01,1002010,P01,1207,0.030,36.21
2020-07-01,1002015,P03,1361,0.025,34.02
2020-07-01,1002013,P01,1298,0.030,38.94


,Account_no,LocationID,CustomerID,ProductID,Billed_usage_kwh,Invoice_date,Base_charge,Price,Bill,Year,Month,MonthName,Quarter,InvoiceMonth
4,1002001,L1001,P01,P01,1389,2020-07-03,12.0,0.03,41.67,2020,7,July,3,2020-07-31
5,1002001,L1001,P01,P01,1335,2020-07-31,12.0,0.03,40.05,2020,7,July,3,2020-07-31
6,1002001,L1001,P01,P01,1712,2020-08-28,12.0,0.03,51.36,2020,8,August,3,2020-08-31


## Part 3 — Aggregate by month & compute totals (8 minutes)
Create a `TotalDue` (Base + Usage*Price), summarize monthly totals, build a customer-by-month pivot, and inspect a single account's history.

In [5]:
# Compute UsageCharge and TotalDue
df["UsageCharge"] = df["Billed_usage_kwh"] * df["Price"]
df["TotalDue"] = df["Base_charge"] + df["UsageCharge"]
display(df[["Billed_usage_kwh","Price","Base_charge","Bill","TotalDue"]].head(5))

# Monthly usage & revenue summary (across all accounts)
monthly_summary = (
    df.groupby("InvoiceMonth")
      .agg(
          Total_kWh=("Billed_usage_kwh","sum"),
          Total_Bill=("Bill","sum"),
          Total_Due=("TotalDue","sum")
      )
      .reset_index()
      .sort_values("InvoiceMonth")
)
display(monthly_summary.head())

# Monthly pivot by CustomerID
customer_month_pivot = (
    df.pivot_table(
        index="InvoiceMonth",
        columns="CustomerID",
        values="TotalDue",
        aggfunc="sum"
    )
    .sort_index()
)
display(customer_month_pivot.head())

# Account history example (pick one account)
example_acct = df["Account_no"].iloc[0]
acct_history = (
    df.loc[df["Account_no"] == example_acct, ["Invoice_date","Billed_usage_kwh","TotalDue"]]
      .sort_values("Invoice_date")
      .reset_index(drop=True)
)
display(acct_history.head())

,Billed_usage_kwh,Price,Base_charge,Bill,TotalDue
0,302,0.03,12.0,9.06,21.06
1,520,0.03,12.0,15.60,27.60
2,619,0.03,12.0,18.57,30.57
3,614,0.03,12.0,18.42,30.42
4,1389,0.03,12.0,41.67,53.67


,InvoiceMonth,Total_kWh,Total_Bill,Total_Due
0,2020-03-31,14086,472.08,832.075
1,2020-04-30,13338,463.05,799.050
2,2020-05-31,10725,354.47,618.465
3,2020-06-30,29303,932.15,1280.150
4,2020-07-31,53580,1814.84,2210.850


CustomerID,P01,P02,P03
InvoiceMonth,,,
2020-03-31,252.87,332.505,246.70
2020-04-30,248.88,363.270,186.90
2020-05-31,148.08,249.735,220.65
2020-06-30,398.25,448.200,433.70
2020-07-31,655.86,1017.090,537.90


,Invoice_date,Billed_usage_kwh,TotalDue
0,2020-03-05,302,21.06
1,2020-04-06,520,27.60
2,2020-05-04,619,30.57
3,2020-06-03,614,30.42
4,2020-07-03,1389,53.67


## Part 4 — Write & read files (CSV + Excel) (5 minutes)
Export analysis to CSVs and a multi-sheet Excel file, then read them back as a quick verification.

In [6]:
# Write CSV outputs
monthly_summary.to_csv("outputs/monthly_summary.csv", index=False)
customer_month_pivot.to_csv("outputs/customer_month_pivot.csv")  # keep index
acct_history.to_csv("outputs/account_history.csv", index=False)
print("Wrote CSVs to ./outputs/")

# Write a multi-sheet Excel workbook
with pd.ExcelWriter("outputs/pandas_lab_outputs.xlsx", engine="openpyxl") as xlw:
    df.to_excel(xlw, sheet_name="RawData", index=False)
    monthly_summary.to_excel(xlw, sheet_name="MonthlySummary", index=False)
    customer_month_pivot.to_excel(xlw, sheet_name="CustomerPivot")
    acct_history.to_excel(xlw, sheet_name="AccountHistory", index=False)
print("Wrote Excel workbook to ./outputs/pandas_lab_outputs.xlsx")

# Read back from Excel to verify
ms_back = pd.read_excel("outputs/pandas_lab_outputs.xlsx", sheet_name="MonthlySummary")
display(ms_back.head())

sheets = pd.read_excel("outputs/pandas_lab_outputs.xlsx", sheet_name=["MonthlySummary","CustomerPivot"])
print(list(sheets.keys()))
display(sheets["CustomerPivot"].head())

Wrote CSVs to ./outputs/
Wrote Excel workbook to ./outputs/pandas_lab_outputs.xlsx


,InvoiceMonth,Total_kWh,Total_Bill,Total_Due
0,2020-03-31,14086,472.08,832.075
1,2020-04-30,13338,463.05,799.050
2,2020-05-31,10725,354.47,618.465
3,2020-06-30,29303,932.15,1280.150
4,2020-07-31,53580,1814.84,2210.850


['MonthlySummary', 'CustomerPivot']


,InvoiceMonth,P01,P02,P03
0,2020-03-31,252.87,332.505,246.70
1,2020-04-30,248.88,363.270,186.90
2,2020-05-31,148.08,249.735,220.65
3,2020-06-30,398.25,448.200,433.70
4,2020-07-31,655.86,1017.090,537.90


## Stretch goals (time permitting)
- **Date formatting for presentation:**
  ```python
  monthly_summary["MonthLabel"] = monthly_summary["InvoiceMonth"].dt.strftime("%b %Y")
  monthly_summary[["MonthLabel","Total_kWh","Total_Due"]].head()
  ```
- **Export filtered subsets** (e.g., only 2020 data) to a new CSV/Excel.
- **Append mode**: Write a second summary sheet to the same Excel file with a different name (e.g., `"Monthly_Usage_Only"`).
- **Input validation**: If `SampleData.csv` is missing, catch `FileNotFoundError` and print a friendly message.


## Instructor quick demo — all-in-one cell (optional)
Run this single cell to generate the major outputs.

In [ ]:
import pandas as pd
from pathlib import Path

Path("outputs").mkdir(exist_ok=True)

df = pd.read_csv("SampleData.csv", parse_dates=["Invoice_date"])
df["Year"] = df["Invoice_date"].dt.year
df["Month"] = df["Invoice_date"].dt.month
df["MonthName"] = df["Invoice_date"].dt.month_name()
df["Quarter"] = df["Invoice_date"].dt.quarter
df["InvoiceMonth"] = df["Invoice_date"].dt.to_period("M").dt.to_timestamp("M")

df["UsageCharge"] = df["Billed_usage_kwh"] * df["Price"]
df["TotalDue"] = df["Base_charge"] + df["UsageCharge"]

monthly_summary = (
    df.groupby("InvoiceMonth")
      .agg(Total_kWh=("Billed_usage_kwh","sum"),
           Total_Bill=("Bill","sum"),
           Total_Due=("TotalDue","sum"))
      .reset_index()
      .sort_values("InvoiceMonth")
)

customer_month_pivot = (
    df.pivot_table(index="InvoiceMonth", columns="CustomerID", values="TotalDue", aggfunc="sum")
    .sort_index()
)

example_acct = df["Account_no"].iloc[0]
acct_history = (
    df.loc[df["Account_no"] == example_acct, ["Invoice_date","Billed_usage_kwh","TotalDue"]]
      .sort_values("Invoice_date")
      .reset_index(drop=True)
)

monthly_summary.to_csv("outputs/monthly_summary.csv", index=False)
customer_month_pivot.to_csv("outputs/customer_month_pivot.csv")
acct_history.to_csv("outputs/account_history.csv", index=False)

with pd.ExcelWriter("outputs/pandas_lab_outputs.xlsx", engine="openpyxl") as xlw:
    df.to_excel(xlw, sheet_name="RawData", index=False)
    monthly_summary.to_excel(xlw, sheet_name="MonthlySummary", index=False)
    customer_month_pivot.to_excel(xlw, sheet_name="CustomerPivot")
    acct_history.to_excel(xlw, sheet_name="AccountHistory", index=False)

print("Done. Files written to ./outputs/")

# Stretch Goal Solutions

These solutions assume you've already read `SampleData.csv` with `parse_dates=["Invoice_date"]` and created `InvoiceMonth` as in the main lab.

## 1) Date formatting for presentation (`MonthLabel`)

In [7]:
# Create MonthLabel on the monthly summary
monthly_summary["MonthLabel"] = monthly_summary["InvoiceMonth"].dt.strftime("%b %Y")
display(monthly_summary[["MonthLabel", "Total_kWh", "Total_Due"]].head(10))

# Or, directly from raw dates (if needed)
df["MonthLabel"] = df["Invoice_date"].dt.strftime("%b %Y")
display(df[["Invoice_date", "MonthLabel"]].head(10))

,MonthLabel,Total_kWh,Total_Due
0,Mar 2020,14086,832.075
1,Apr 2020,13338,799.050
2,May 2020,10725,618.465
3,Jun 2020,29303,1280.150
4,Jul 2020,53580,2210.850
5,Aug 2020,39637,1609.565
6,Sep 2020,12824,724.140
7,Oct 2020,12416,709.775
8,Nov 2020,26386,1159.545
9,Dec 2020,27688,1216.960


,Invoice_date,MonthLabel
0,2020-03-05,Mar 2020
1,2020-04-06,Apr 2020
2,2020-05-04,May 2020
3,2020-06-03,Jun 2020
4,2020-07-03,Jul 2020
5,2020-07-31,Jul 2020
6,2020-08-28,Aug 2020
7,2020-09-28,Sep 2020
8,2020-10-27,Oct 2020
9,2020-11-26,Nov 2020


## 2) Export 2020-only rows to new CSV/Excel

In [8]:
# A) Filter by dt.year
df_2020 = df[df["Invoice_date"].dt.year == 2020].copy()
df_2020.to_csv("outputs/only_2020.csv", index=False)
df_2020.to_excel("outputs/only_2020.xlsx", index=False, sheet_name="Only2020")

# B) Filter by between on Invoice_date (alternative)
start, end = "2020-01-01", "2020-12-31"
df_2020_alt = df[(df["Invoice_date"] >= start) & (df["Invoice_date"] <= end)].copy()
df_2020_alt.to_csv("outputs/only_2020_between.csv", index=False)

# C) Using normalized InvoiceMonth boundaries
df_2020_months = df[df["InvoiceMonth"].between("2020-01-31", "2020-12-31")].copy()
df_2020_months.to_excel("outputs/only_2020_months.xlsx", index=False, sheet_name="Only2020")

## 3) Append an additional sheet to the same Excel file

In [9]:
# Example: append a Monthly_Usage_Only sheet to outputs/pandas_lab_outputs.xlsx
monthly_usage_only = (
    df.groupby("InvoiceMonth")
      .agg(Total_kWh=("Billed_usage_kwh", "sum"))
      .reset_index()
      .sort_values("InvoiceMonth")
)

with pd.ExcelWriter(
    "outputs/pandas_lab_outputs.xlsx",
    engine="openpyxl",
    mode="a",
    if_sheet_exists="replace"  # replace if the sheet already exists
) as xlw:
    monthly_usage_only.to_excel(xlw, sheet_name="Monthly_Usage_Only", index=False)

print("Appended Monthly_Usage_Only to outputs/pandas_lab_outputs.xlsx")

Appended Monthly_Usage_Only to outputs/pandas_lab_outputs.xlsx


## 4) Input validation: friendly errors for missing CSV & bad dates

In [11]:
import pandas as pd
from pathlib import Path

csv_path = Path("SampleData.csv")
try:
    if not csv_path.exists():
        raise FileNotFoundError(f"Could not find {csv_path.resolve()}.")

    df_valid = pd.read_csv(csv_path, parse_dates=["Invoice_date"])  # new var to avoid overwriting
    df_valid["Invoice_date"] = pd.to_datetime(df_valid["Invoice_date"], errors="coerce")
    bad_dates = df_valid["Invoice_date"].isna().sum()
    if bad_dates > 0:
        raise ValueError(f"{bad_dates} row(s) have invalid or missing Invoice_date. Please check the source file.")

    print("CSV loaded successfully for validation.")
    print("Rows:", len(df_valid), "| Columns:", len(df_valid.columns))
except FileNotFoundError as e:
    print("ERROR:", e)
    print("Hint: Place 'SampleData.csv' in the same folder as this notebook/script.")
except ValueError as e:
    print("ERROR:", e)
    # Optionally write problematic rows
    # df_valid[df_valid["Invoice_date"].isna()].to_csv("outputs/bad_date_rows.csv", index=False)
except Exception as e:
    print("Unexpected error while reading SampleData.csv:", repr(e))

ERROR: Could not find /home/sysadmin/source/repos/extreme-python/solutions/SampleData.csv.
Hint: Place 'SampleData.csv' in the same folder as this notebook/script.


## Optional: Bundle the stretch goals end-to-end

In [ ]:
# 1) MonthLabel on summary
monthly_summary["MonthLabel"] = monthly_summary["InvoiceMonth"].dt.strftime("%b %Y")

# 2) 2020-only exports (CSV + Excel)
df_2020 = df[df["Invoice_date"].dt.year == 2020].copy()
df_2020.to_csv("outputs/only_2020.csv", index=False)
df_2020.to_excel("outputs/only_2020.xlsx", index=False, sheet_name="Only2020")

# 3) Append extra sheet to the main Excel file
monthly_usage_only = (
    df.groupby("InvoiceMonth")
      .agg(Total_kWh=("Billed_usage_kwh", "sum"))
      .reset_index()
      .sort_values("InvoiceMonth")
)
with pd.ExcelWriter(
    "outputs/pandas_lab_outputs.xlsx",
    engine="openpyxl",
    mode="a",
    if_sheet_exists="replace"
) as xlw:
    monthly_usage_only.to_excel(xlw, sheet_name="Monthly_Usage_Only", index=False)

print("Stretch goals completed: MonthLabel + 2020 exports + appended sheet.")